# Lab.09 / IBM3202 – Simulaciones de Plegamiento utilizando Modelos Basados en Estructura


### Aspectos Teóricos

Si bien las dinámicas moleculares convencionales (MD) son muy útiles para predecir el comportamiento dinámico de biomoléculas o profundizar nuestra comprensión de observaciones experimentales, su alto costo computacional introduce **limitaciones en las escalas de tiempo de los movimientos que se pueden explorar** (típicamente en el orden de **ns – μs**). En contraste, los procesos biológicos de mayor complejidad como el **plegamiento de proteínas, cambios conformacionales e interacciones biomoleculares**, están en el orden de **ms – s**. Estos fenómenos no pueden ser estudiados a través de estrategias tradicionales de MD, por lo que se requiere la aplicación de técnicas de muestreo mejorado para alcanzar escalas de tiempo biológicamente relevantes.

En este sentido, **la teoría del paisaje de energía del plegamiento de proteínas** y el **principio de mínima frustración** han proporcionado la base para la definición de una función de energía potencial simplificada que nos permite explorar estos procesos. Este marco establece que **la robustez del plegamiento de proteínas durante la evolución es el resultado de la selección de secuencias que acumulan interacciones que se apoyan mutuamente en el estado nativo de una proteína**, evitando la acumulación de interacciones conflictivas (frustrantes) para la estabilidad de este estado plegado.

Estas interacciones están fácilmente **disponibles en el estado nativo resuelto por cristalografía de rayos X o RMN** para una proteína dada y se pueden extraer mediante un criterio basado en distancias (**Fig. 1**). En un escenario donde todas las interacciones presentes en el estado nativo son atractivas, el **paisaje de energía de plegamiento es canalizado**. Estos conceptos teóricos constituyen el argumento central para la generación y aplicación de **Modelos Basados en Estructura** (SBM) en simulaciones de MD.


<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_01.png'/>
<figcaption>FIGURA 1. Utilizando un criterio basado en distancias para determinar contactos nativos basados en la estructura de una proteína, aquí la proteína ribosomal S6. El panel izquierdo muestra la proximidad del contacto atómico más cercano para cada par de residuos hasta un máximo de 1.5 nm, mientras que el panel derecho muestra el mapa de contactos nativos a nivel grueso resultante al filtrar por un umbral de distancia de 0.6 nm y una separación mínima de secuencia de 3 residuos entre residuos interactuantes.
<br> Noel JK & Onuchic JN (2012) <i> Computational Modeling of Biological Systems, 31-54</i></figcaption></center>
</figure>

**Los modelos SBM incluyen explícitamente un sesgo nativo en la función de energía potencial**, considerando todos los contactos entre residuos cercanos espacialmente **en la estructura nativa** de una proteína como **interacciones atractivas**, mientras que todas las interacciones no nativas (es decir, aquellas que no se obtienen mediante el análisis de la estructura nativa) se tratan como repulsivas. Esta condición es consistente con la noción de una secuencia de una proteína sin interacciones altamente frustrantes (es decir, un paisaje de plegamiento canalizado). En este contexto, tanto el mecanismo de plegamiento como las barreras de energía resultan principalmente de las restricciones geométricas impuestas por la conectividad de la cadena polipeptídica y su empaquetamiento en el espacio tridimensional, o **topología**.

Además, los modelos SBM también reducen la complejidad del sistema biomolecular utilizando una aproximación a nivel grueso, en la cual cada residuo se reemplaza por una perla centrada en las coordenadas del carbono alfa. Sin embargo, los modelos SBM más recientes también son capaces de utilizar **representaciones a nivel de átomos**.


La función de energía potencial que define las interacciones enlazadas y no enlazadas de los modelos SBM incluye la información sobre la topología de la estructura nativa de la siguiente manera:

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_02.png' />
</center>
</figure>

Donde $\varepsilon_{r} = 100\varepsilon$, $\varepsilon_{\theta} = 20 \varepsilon$, $\varepsilon_{D} = \varepsilon$, $\varepsilon_{C} = \varepsilon_{NC} = \varepsilon = k_{B}T$, que es el producto entre la constante de Boltzmann y la temperatura en unidades reducidas; $r_{0}$, $\theta_{0}$ y $F_{D}(\phi_{0})$ son los enlaces, ángulos y torsiones de la estructura nativa y se mantienen mediante restricciones armónicas; $\sigma_{ij}$ es la distancia entre los residuos i y j en el estado nativo (calculado a partir de la estructura nativa) y $\sigma_{NC}$ = 4 Å.

Mientras que los contactos nativos se tratan como interacciones atractivas a través de un potencial de Lennard-Jones (LJ) de 12-10, los contactos no nativos se tratan como repulsivos.
De esta manera, cada vez que la temperatura de la simulación sea menor que la **temperatura de plegamiento**, $T_{F}$, el sistema tenderá a la energía mínima, en este caso definida como la estructura del estado nativo.


##Aspectos experimentales

En este tutorial simularemos el plegamiento de un pequeño dominio de proteína SH3 a partir de la estructura PDB 1FMK utilizando un SBM a nivel grueso y GROMACS.

Para ello, primero generaremos un modelo SBM a nivel grueso usando **SMOG2** y luego prepararemos y ejecutaremos los archivos de simulación en una versión de GROMACS mejorada con SBM. Finalmente, analizaremos los resultados en términos de las fluctuaciones estructurales y energéticas del sistema simulado, con la esperanza de determinar su paisaje de energía libre utilizando el **Método de Análisis de Histograma Ponderado (WHAM)**.


#Parte 0. Descarga e instalación

Antes de comenzar, **recuerda iniciar el entorno de ejecución hospedado** en Google Colab.

Luego, debemos instalar varios programas para realizar este tutorial, a saber:
- **biopython** para la manipulación de archivos PDB.
- **py3Dmol** para la visualización de la estructura de proteínas.
- **cpanm** para la instalación de varias utilidades de Perl necesarias para ejecutar SMOG2.
- **SMOG2** para generar nuestros modelos basados en estructura.
- **GROMACS mejorado con SBM** para preparar nuestro sistema MD y realizar nuestras simulaciones MD.

Para visualizar nuestras trayectorias MD, utilizaremos una versión web de **NGLview**. Esto se debe a la incapacidad de Google Colab para manejar un paquete de Python necesario para cargar NGLview directamente en Google Colab. Con suerte, esto cambiará en el futuro cercano.

1. Primero, comenzaremos descargando y configurando SMOG2 en Google Colab, lo que requiere la instalación de varias utilidades de Perl mediante cpanminus.

**NOTA**: Esta instalación tarda ~10 minutos. Si es posible, realiza esta instalación antes de que comience la sesión del tutorial.


In [ ]:
#Descarga y extracción de SMOG2 desde SMOG-server
!wget https://smog-server.org/smog2/code/smog-2.4.2.tgz
!tar zxf smog-2.4.2.tgz

In [ ]:
#Automatic configuration of cpan for Perl
!echo y | cpan
!cpan App::cpanminus
#Instalando los programas Perl necesarios para SMOG2
!cpanm String::Util #--local-lib $nb_path
!cpanm XML::Simple #--local-lib $nb_path
!cpanm Exporter #--local-lib $nb_path
!cpanm PDL #--local-lib $nb_path
!cpanm XML::Validator::Schema #--local-lib $nb_path
# New Perl module added: requirement for smog v2.4.2
!cpanm XML::LibXML #--local-lib $nb_path

In [ ]:
#Descargando un SMOG2 previamente configurado
%%bash
rm -r /content/smog-2.4.2/configure.smog2
wget -P /content/smog-2.4.2 https://github.com/pb3lab/ibm3202/raw/master/software/configure.smog2
source /content/smog-2.4.2/configure.smog2
smog2 -h

2. Luego, configuraremos nuestro GROMACS mejorado con SBM en Google Colab, basado en una versión previamente compilada e instalada de GROMACS.


In [ ]:
# descarga y descompresión de SBM-enhanced GROMACS
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs_sbm.tar.gz
!tar xzf gromacs_sbm.tar.gz

3. Instalación de bio y py3Dmol

In [ ]:
!pip install biopython py3dmol

Una vez completados estos procesos de instalación de software, estamos listos para realizar nuestros experimentos.


# Parte I – Generar un modelo de grano grueso SBM usando SMOG2

Para este paso, necesitamos un conjunto inicial de coordenadas atómicas, que generalmente provienen de una estructura de proteína descargada del Banco de Datos de Proteínas (PDB). Una vez descargados, estos archivos PDB deben limpiarse de moléculas de agua (en el caso de estructuras cristalinas) o debe seleccionarse un solo modelo de las muchas soluciones para un solo conjunto de desplazamientos químicos (en el caso de estructuras de RMN). Por último, estas coordenadas se procesan con SMOG2 para generar nuestro SBM a nivel grueso, en el que cada residuo se representa con una perla centrada en las coordenadas de su átomo C${\alpha}$ correspondiente (por lo tanto, denominado **modelo CA**).

1. Comenzaremos creando y accediendo a una carpeta para preparar nuestro sistema (¿recuerdas nuestro tutorial anterior?).


In [ ]:
# Creemos primero una carpeta. Necesitamos importar las bibliotecas os y path
import os
from pathlib import Path

# Luego, definimos la ruta de la carpeta que queremos crear.
# Observa que la carpeta HOME para un entorno de ejecución hospedado en colab es /content/
smogpath = Path("/content/prepare_1FMK/")

# Ahora, creamos la carpeta usando el comando os.mkdir()
# La condición if es solo para verificar si la carpeta ya existe
# En ese caso, Python devuelve un error
if os.path.exists(smogpath):
  print("La ruta ya existe")
if not os.path.exists(smogpath):
  os.mkdir(smogpath)
  print("La ruta se creó correctamente")

In [ ]:
#cambiando de directorio
os.chdir(smogpath)

2. Luego, descargaremos la estructura resuelta de la tirosina quinasa humana de longitud completa (ID del PDB 1FMK). Esta estructura de proteínas resuelta contiene varios dominios, pero solo nos interesa su dominio SH3. Por lo tanto, volveremos a usar `Dice` de biopython no solo para eliminar moléculas de agua y ligandos, sino también para seleccionar los residuos que corresponden al dominio SH3.

**PREGUNTA❓**: Al observar los scripts a continuación, ¿qué residuos componen el dominio SH3 de la tirosina quinasa humana?


In [ ]:
#importa el PDB
import os
from Bio.PDB import *
pdbid = ['1fmk']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True)
  os.rename("pdb"+s+".ent", s+".pdb")

In [ ]:
#dejamos un parser para el PDB
parser = PDBParser()
io=PDBIO()
structure = parser.get_structure('X', '1fmk.pdb')
#eliminamos aguas, ligandos e hidrógenos con DICE
io.set_structure(structure)
sel = Dice.ChainSelector('A', 82, 145)
io.save("1fmk_clean.txt", sel)
print("Tu archivo PDB ha sido procesado. ¡Solo se han conservado los átomos pesados de la proteína!")
print("Se ha guardado como un archivo de texto para editarlo en Google Colab")
print("Recuerda editarlo antes de usar SMOG2")

3. Visualicemos la estructura con py3Dmol

In [ ]:
```python
import py3Dmol

# Primero asignamos py3Dmol.view como view
view = py3Dmol.view()

# Las siguientes líneas se utilizan para agregar la clase addModel
# para leer los archivos PDB
view.addModel(open('1fmk_clean.txt', 'r').read(), 'pdb')

# Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')

# Aquí establecemos el estilo y color de la visualización
view.setStyle({'chain':'A'}, {'cartoon': {'color':'spectrum'}})

# Aquí centramos la molécula para su visualización
view.zoomTo()

# Y finalmente visualizamos las estructuras con el siguiente comando
view.show()

4. Como notaste en el script anterior, generamos el PDB final como un archivo de texto **.txt**. Esto se debe a que estas estructuras requieren cierta edición en Google Colab antes de ser procesadas por SMOG2:

   - Si hay un residuo de **extremo C-terminal** presente, debemos renombrarlo agregando un carácter **`T`** justo después del nombre del residuo, para todos los átomos que comprende dicho residuo C-terminal. Esto es necesario para que SMOG2 reconozca los extremos C-terminales.
      - Ejemplo:
```
# Before editing
ATOM    504  N   ALA A 145...
# After editing
ATOM    504  N   ALATA 145...
```
   - **Eliminar líneas `TER`** del PDB final y **usarlas solo como separadores entre cadenas en proteínas oligoméricas**.
      - Ejemplo:
```
# Before editing
ATOM    508  CB  ALA A 145...
TER     509      ALA A 145
END
# After editing
ATOM    508  CB  ALA A 145...
END
```

Corrige tu PDB de acuerdo a esto.

**Pregunta❓**: Necesitamos editar nuestro PDB para asignar un terminal Carboxilo en nuestro caso?

5. Una vez hecho esto, podemos procesar nuestro archivo en SMOG2 como se indica a continuación:

In [ ]:
%%bash
source /content/smog-2.4.2/configure.smog2
smog2 -i 1fmk_clean.txt -CA -dname SH3_smog

6. Examina los archivos SBM generados. Los archivos más relevantes se describen detalladamente a continuación (y puedes leer esto mientras ejecutas tus simulaciones).

- **Archivo de coordenadas .gro** – Archivo compatible con GROMACS que contiene el número total de átomos y las coordenadas de cada átomo en {x, y, z}
  ![.gro coordinate file](https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_03.png)
  
  (**PREGUNTA❓:** ¿hay algo particular con este archivo .gro?)

- **Archivo de topología .top** – Contiene la topología o interacciones enlazadas y no enlazadas del sistema. Todos estos parámetros se extraen de la estructura nativa definida en el archivo PDB.
  ![.top topology file](https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/smog_04.png)
  
  - La composición atómica del sistema se describe en la sección `[atoms]`
  - Los enlaces covalentes se enumeran en la sección `[bonds]`
  - Los ángulos entre 3 átomos conectados por enlaces covalentes se enumeran en la sección `[angles]`
  - Los ángulos diedros entre 4 átomos conectados por enlaces covalentes (y también configuraciones planas para cadenas laterales aromáticas) se enumeran en la sección `[dihedrals]`
  - Los contactos nativos no enlazados se indican en la sección `[pairs]`
  - Las interacciones no enlazadas no nativas se tratan como repulsivas al enumerar los contactos nativos en la sección `[exclusions]`; en otras palabras, lo que no se enumera en esta sección es no nativo.

- **Archivo de índices .ndx** – Enumera grupos de átomos que constituyen las diversas cadenas registradas en el archivo .pdb (por ejemplo, dos o más cadenas de un oligómero)

- **Archivo de mapa de contactos .contacts.CG** – Archivo de 4 columnas, en el que las columnas 2 y 4 contienen los números de átomos en contacto en el estado nativo, y las columnas 1 y 3 corresponden a los identificadores de los grupos en el archivo .ndx (es decir, la cadena a la que pertenece cada átomo).


7. Visualicemos nuestra estructura!

In [ ]:
import py3Dmol

# Primero asignamos py3Dmol.view como view
view = py3Dmol.view()

# Las siguientes líneas se utilizan para añadir la clase addModel
# para leer archivos GRO
view.addModel(open('SH3_smog.gro', 'r').read(), 'gro')

# Aquí establecemos el color de fondo como blanco
view.setBackgroundColor('white')

# Aquí establecemos el estilo de visualización y color
view.setStyle({'esfera': {'esquemaColores': {'propiedad': 'serial', 'gradiente': 'sinebow', 'min': 1, 'max': 64}}})

# Aquí centramos la molécula para su visualización
view.zoomTo()

# Y finalmente visualizamos las estructuras usando el siguiente comando
view.show()


Como puedes ver aquí, nuestra estructura está ahora **a grano grueso** (**coarse-grained**), con cada residuo representado por una sola esfera centrada en el C${\alpha}$ de la estructura original en formato PDB de átomos completos.


# Parte II - Preparar y ejecutar las simulaciones de plegamiento SBM en GROMACS

Ahora estamos listos para realizar nuestras simulaciones utilizando estos modelos SBM.

1. Comenzaremos creando una nueva carpeta para preparar y ejecutar nuestras simulaciones de dinámica molecular (MD), en la cual copiaremos nuestros archivos de coordenadas y topología SBM.


In [ ]:
# Definiendo una nueva carpeta para las simulaciones de MD
mdpath = Path("/content/md_1FMK/")

# Ahora creamos la carpeta usando el comando os.mkdir()
# La condición if es solo para verificar si la carpeta ya existe
# En caso afirmativo, Python devuelve un error
if os.path.exists(mdpath):
    print("La ruta ya existe")
if not os.path.exists(mdpath):
    os.mkdir(mdpath)
    print("La ruta se creó exitosamente")

In [ ]:
# Cambiando al directorio recién creado y copiando los archivos .gro y .top
os.chdir(mdpath)
from shutil import copyfile
copyfile(smogpath/'SH3_smog.gro', mdpath/'SH3_smog.gro')
copyfile(smogpath/'SH3_smog.top', mdpath/'SH3_smog.top')

2. Para GROMACS, también necesitaremos un **archivo de instrucciones de MD**, como en nuestro tutorial anterior. Descargaremos dicho archivo desde GitHub (**mdrun_CA_v5.mdp**), el cual contiene los parámetros de las simulaciones de dinámica molecular (MD), como el integrador de la posición y velocidad de los átomos para cada tiempo de simulación; el paso de integración temporal; el tiempo total de simulación; la frecuencia de escritura de los archivos de salida; las distancias umbrales para el tratamiento de las interacciones nativas; la temperatura de la simulación; etc.

  **ADVERTENCIA⚠️:** Se recomienda encarecidamente leer los comentarios e instrucciones contenidos en este archivo, ya que difiere sustancialmente de las instrucciones utilizadas en una MD convencional.

  **PREGUNTA❓:** ¿A qué temperatura estamos ejecutando nuestras simulaciones?


In [ ]:
!wget https://github.com/pb3lab/ibm3202/raw/master/files/mdrun_CA_v5.mdp

Los parámetros más importantes en este archivo .mdp son la **temperatura de la simulación**; la **temperatura para las velocidades iniciales**; la distancia de corte para evaluar las interacciones; y la llamada a **tablas definidas por el usuario** para nuestras interacciones no enlazadas (lo cual se explica a continuación).

3. Internamente, GROMACS tiene la capacidad de utilizar potenciales de Lennard-Jones (LJ) de tipo 12-6, mientras que el potencial requerido para los contactos nativos en simulaciones SBM utilizando modelos CA corresponde a LJ 12-10. Por lo tanto, descargaremos un script en Perl para generar una **tabla definida por el usuario** de potenciales tabulados LJ 12-10. El resultado de ejecutar este script es un archivo **.xvg** que contiene el potencial tabulado.


In [ ]:
%%bash
wget https://github.com/pb3lab/ibm3202/raw/master/files/maketable4.pl
perl maketable4.pl > table.xvg

4. Ahora que tenemos todos los archivos que necesitamos, primero crearemos una carpeta con el nombre de nuestra temperatura de simulación, en la cual ejecutaremos nuestras simulaciones.


In [ ]:
# Creando una carpeta para ejecutar una simulación a una temperatura específica
temppath = Path("152")

# Ahora, creamos la carpeta usando el comando os.mkdir()
# La condición if es solo para verificar si la carpeta ya existe
# En caso afirmativo, Python devuelve un error
if os.path.exists(temppath):
    print("La ruta ya existe")
if not os.path.exists(temppath):
    os.mkdir(temppath)
    print("La ruta se creó exitosamente")

5. Por último, prepararemos nuestro archivo de entrada de ejecución binaria portable (.tpr) para GROMACS en esta carpeta y ¡ejecutaremos nuestra simulación! Ten en cuenta cómo le indicamos a GROMACS que utilice nuestra tabla personalizada de potenciales tabulados LJ 12-10.

Esta simulación tarda ~30 min. **Reduce el número de pasos según tus necesidades para este tutorial.**


In [ ]:
#Cambiando directorio
os.chdir(temppath)

In [ ]:
#Preparando la precompilación
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx grompp -f ../mdrun_CA_v5.mdp -c ../SH3_smog.gro -p ../SH3_smog.top -o run.tpr

In [ ]:
#Corriendo la simulación
%%time
%%bash
source /content/gromacs_sbm/bin/GMXRC
gmx mdrun -s run.tpr -table ../table.xvg -tablep ../table.xvg -nt 2 -noddcheck

Una vez que nuestra simulación esté completa, ¡estamos listos para analizarla!

**Pero, ¿qué hay de la temperatura?** Algo a tener en cuenta es que esta temperatura no representa unidades físicas típicas, sino unidades reducidas (es decir, ponderadas por Boltzmann), donde $T^* = \varepsilon = k_BT$. Esto es necesario para un análisis energético adicional de las simulaciones. Para comprender mejor estas unidades reducidas, consulta las preguntas frecuentes del servidor SMOG.


#Parte III – Analizando la simulación

Una vez que nuestras simulaciones estén completas, podemos analizar cómo fluctúa la estructura de la proteína y la energía potencial a lo largo de la trayectoria.


1. Primero, utilizaremos las utilidades de GROMACS para obtener los cambios en RMSD y energía potencial a lo largo de la trayectoria. Extraeremos el RMSD utilizando el módulo `rmsd` y la energía potencial utilizando el módulo `energy`.

Ten en cuenta cómo proporcionamos los comandos apropiados a GROMACS para obtener estos resultados.


In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
# Comandos para RMSD
echo "0" > options
echo " " >> options
echo "0" >> options
echo " " >> options
# Cálculo de RMSD
gmx rms -s ../SH3_smog.gro -f traj_comp.xtc -xvg none < options
# Comandos para energía potencial
echo "Potential" > options
echo " " >> options
# Extracción de energía potencial
gmx energy -f ener.edr -xvg none < options

2. Una métrica mejor para observar las fluctuaciones en la estructura de nuestra proteína durante la simulación es mediante el cálculo de los cambios en los contactos nativos a lo largo de la trayectoria. Para esto, utilizaremos el módulo `g_kuh`, que está disponible exclusivamente en la versión mejorada de GROMACS para SBM.

Para que `g_kuh` se ejecute, primero necesitamos un archivo de índice **.ndx** que contenga un título seguido de la lista de pares de átomos en contacto en el estado nativo, como se muestra a continuación:

```
[ contacts ]
atom_i atom_j
...
```
Podemos generar este archivo imprimiendo las columnas 2 y 4 del archivo **.contacts.CG** generado por SMOG2:


In [ ]:
f = open(mdpath/"contacts.ndx", "a")
f.write("[ contacts ]\n")
with open(smogpath/"SH3_smog.contacts.CG") as infile:
  for line in infile:
    if line.strip():
      cols = line.split()
      f.write(cols[1] + "\t" + cols[3] + "\n")
f.close()

3. Luego, podemos ejecutar `g_kuh` para calcular el número de contactos nativos para cada coordenada guardada durante la simulación de MD en el archivo de trayectoria comprimido **.xtc**.

En este caso, la opción `-s` indica nuestra estructura de proteína inicial a partir de la cual se calcularán las distancias de contacto nativas, la opción `-n` indica nuestro archivo de contacto, las opciones `-noabscut` y `-cut` indican que `-cut` se toma como la desviación relativa máxima de la distancia nativa para un contacto formado (en este caso, un 20% más largo que la distancia nativa), y `-noshortcut` especifica que solo se consideran desviaciones a distancias mayores que el valor nativo.


In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
g_kuh -s ../SH3_smog.gro -f traj_comp.xtc -n ../contacts.ndx -noabscut -noshortcut -cut 0.2

4. ¡Vamos a graficar nuestros resultados y ver qué sucedió durante nuestra simulación! Graficaremos primero el cambio en los contactos nativos (Q), y luego el cambio en la energía potencial. Puedes verificar el cambio en RMSD por ti mismo.


In [ ]:
!paste rmsd.xvg qvals.out > data.txt
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('data.txt')

plt.title('Structural fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('Q')
plt.plot(data[:,0], data[:,2], linestyle='solid', linewidth='2', color='red')
plt.show()

In [ ]:
data = np.loadtxt('energy.xvg')

plt.title('Potential energy fluctuations of the system')
plt.xlabel('Time (tau)')
plt.ylabel('Energy')
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

Si las simulaciones se configuraron correctamente a la temperatura de plegamiento para esta proteína, deberías observar **tanto el estado nativo** (alto Q, baja energía) **como el desplegado** (bajo Q, alta energía) **de la proteína en proporciones equivalentes, así como varias transiciones entre estos estados**.


5. Para finalizar, visualizaremos nuestra simulación. Para ello, utilizaremos el módulo `trjconv` para extraer solo la proteína de nuestro sistema y convertir nuestra trayectoria en un archivo PDB. Luego, descargaremos este nuevo archivo PDB y lo cargaremos en [**NGLviewer**](http://nglviewer.org/ngl/) como un archivo PDB de **trayectoria**.


In [ ]:
%%bash
source /content/gromacs_sbm/bin/GMXRC
#Un truco para dar opciones interactivas a GROMACS
echo "Protein" > options
echo "Protein" >> options
echo " "
gmx trjconv -s run.tpr -f traj_comp.xtc -o traj.pdb < options

In [ ]:
#Descarga
from google.colab import files
files.download("/content/md_dualAKE/traj.pdb")

**¡Y esto es el final del noveno tutorial!**

Si deseas descargar tus resultados, puedes realizar los siguientes comandos:


In [ ]:
os.chdir("/content/")
!zip -r smogmd.zip $mdpath
!zip -r smog1FMK.zip $smogpath
from google.colab import files
files.download("/content/smogmd.zip")
files.download("/content/smog1FMK.zip")

# **TAREA📚:** Una vez que hayas llegado hasta aquí, ¡puedes realizar algunos ejercicios!

1. Ejecuta algunas simulaciones a diferentes temperaturas, tanto por debajo como por encima de la que se utilizó en este tutorial. ¿Qué esperas si reduces/aumentas la temperatura de la simulación?
2. Ejecuta simulaciones más largas. El análisis del plegamiento de proteínas requiere varias transiciones entre los estados nativo y desplegado y cualquier estado intermedio que pueda estar presente en la ruta del plegamiento de la proteína.


#Apéndice - determinando el paisaje de plegamiento de tu proteína

Normalmente, se logra un muestreo suficiente mediante la ejecución de varias simulaciones de dinámica molecular (MD) prolongadas utilizando estos modelos SBM a diferentes temperaturas por encima y por debajo de la temperatura de plegamiento de la proteína de interés.

 Una vez que se logra esto, la barrera de energía libre que separa los estados desplegado y plegado de tu proteína (así como la presencia de cualquier intermediario) en función de los contactos nativos (o cualquier otro coordenada de reacción) se puede determinar utilizando el **Método de Análisis de Histogramas Ponderados (WHAM, por sus siglas en inglés)**. En resumen, este método consiste en describir la densidad de los estados del sistema en función de la energía y una coordenada de reacción adecuada (como el número de contactos nativos), a través de una aproximación de histogramas ponderados por $k_BT$, donde $k_B$ corresponde a la **constante de Boltzmann**.


1. Primero necesitamos generar archivos que contengan el cambio en la energía potencial y en el número de contactos nativos (Q) a lo largo de la trayectoria en la primera y segunda columnas, respectivamente. Para esto, podemos utilizar los comandos `paste` y `awk` de `bash`.


In [ ]:
!paste energy.xvg qvals.out | awk '{print $2, $3}' > hist.152

2. Realizamos la misma operación para todas las demás temperaturas, asegurándonos de cambiar el nombre del archivo de acuerdo con la temperatura que estamos utilizando (el ejemplo anterior se llama hist.152 ya que corresponde a T = 152), y guardamos todos los archivos generados en una sola carpeta (por ejemplo, analysis). Se puede descargar un ejemplo de carpeta desde GitHub:

In [ ]:
#Volviendo a la carpeta principal
import os
os.chdir("/content/")
!wget https://github.com/pb3lab/ibm3202/raw/master/files/WHAMexample.tar.gz
!tar zxf WHAMexample.tar.gz

3. También descargamos una aplicación Java llamada **WHAM.jar** para realizar el análisis WHAM mediante el uso de un archivo de configuración, que también está incluido en la carpeta descargada anteriormente.


In [ ]:
os.chdir("/content/WHAMexample")
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/WHAM.jar

El archivo de configuración (**por favor revísalo**) contiene información sobre el rango de temperatura para el cual se calculará la capacidad calorífica y la energía libre, así como el espaciado o particionamiento de las coordenadas de reacción (WHAM es un análisis de histograma, de ahí el uso de compartimentos de energía y Q). Además, al final del archivo de configuración, se debe indicar el número de archivos a analizar y los nombres de archivo que se leerán con sus temperaturas correspondientes en el siguiente formato:


```
# Ejemplo
numfiles X #Se indica el número de archivos
name hist.152 temp 152
```
4. Una vez que hayas modificado este script agregando el número, los nombres y las temperaturas correspondientes de los datos que analizarás, puedes ejecutar WHAM con el siguiente comando:



In [ ]:
!java -jar WHAM.jar --config config

5. Si todo va bien, deberías obtener un archivo llamado **cv**, que corresponde al cambio en la capacidad calorífica durante el proceso de desplegamiento, y una serie de archivos **freeXXXX** que son las energías libres en función del número de contactos nativos calculados a diferentes temperaturas (T = XXX.X). ¡Vamos a graficar estos resultados!


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('cv')

plt.title('Heat capacity')
plt.xlabel('Temperature (reduced units)')
plt.ylabel('C$_V$')
plt.xlim(144, 160)
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.loadtxt('free1505')

plt.title('Free energy landscape')
plt.xlabel('Q')
plt.ylabel('F(Q) [$k_BT$]')
plt.xlim(0, 182)
plt.ylim(0, 9)
plt.plot(data[:,0], data[:,1], linestyle='solid', linewidth='2', color='red')
plt.show()

Si analizáramos las simulaciones generadas en este tutorial, el número de transiciones observadas sería demasiado pequeño y la convergencia de estos datos sería cuestionable. Sin embargo, la repetición de estas simulaciones durante períodos más largos y la recopilación de datos a varias temperaturas entre $0.9 > T_{F} > 1.1$, donde $T_{F}$ es la temperatura de plegamiento (~151), generarían resultados similares a los presentados anteriormente.
